# **Selección de Algoritmos** 
Comenzaremos con modelos base simples y progresaremos hacia algoritmos de ensamble más complejos para establecer un benchmark de rendimiento.
# **Entrenamiento Sistemático** 
Crearemos un framework reutilizable para entrenar cada modelo, garantizando que todos sean evaluados bajo las mismas condiciones.
# **Ajuste de Hiperparámetros** 
Optimizaremos el mejor modelo candidato para exprimir su máximo potencial predictivo.

# 1.- **Importar librerias**

In [ ]:
# Importar librerías estándar
import sys
import os

# 2. Configurar la ruta para que Python encuentre el código en 'src'. Esto añade la carpeta raíz del proyecto al sys.path
ruta_raiz_proyecto = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(ruta_raiz_proyecto)

# 3. Importar las funciones directamente.
from src.cargar_analisis import cargar_dataframe, crear_listas_variables
from src.pipelines import preparar_datos_para_modelado
from src.modelos import obtener_configuraciones_modelos, optimizar_y_comparar_modelos, guardar_artefactos_ml

print("Funciones importadas exitosamente desde src/")

Funciones importadas exitosamente desde src/


# **2.- Importar datos**

In [2]:
# Ruta de la base de datos
path_data = '../../data/obesity_estimation_model.csv'

# Ejecutar función para cargar datos
df = cargar_dataframe(path_data)

# Ejecutar la función para generar listas con las variables numericas, categoricas y objetivo
variables_numericas, variables_categoricas, variable_objetivo = crear_listas_variables(to_lower = 1, exclude_mixed = 1)

Archivo CSV cargado exitosamente desde: ../../data/obesity_estimation_model.csv


# **3. Preprocesamiento con Pipelines**

In [3]:
X_train, X_test, y_train, y_test, preprocesador = preparar_datos_para_modelado(df, variable_objetivo, 0.2, 1)

Iniciando la preparación de datos para el modelado...
Datos divididos: 1556 para entrenamiento, 390 para prueba.

Preprocesador creado y ajustado a los datos de entrenamiento exitosamente.


# **4.- Construcción y Ajuste y Evaluación de Modelos**

In [5]:
# Ejecutar la función para obtener los modelos a evaluar
configuracion_modelos = obtener_configuraciones_modelos()

print(f"Se cargaron {len(configuracion_modelos)} configuraciones de modelos.")
print("\nEjemplo: Parámetros de Regresión Logistica:")
print(configuracion_modelos[0]['parametros'])

Se cargaron 7 configuraciones de modelos.

Ejemplo: Parámetros de Regresión Logistica:
{'clasificador__penalty': ['l1', 'l2'], 'clasificador__solver': ['liblinear', 'saga', 'lbfgs'], 'clasificador__C': [0.01, 0.1, 1.0, 10.0]}


In [ ]:
# Ejecutar el proceso de optimización y comparación
df_resultados_opt, pipelines_optimizados = optimizar_y_comparar_modelos(
    configuracion_modelos,
    preprocesador,
    X_train, y_train, X_test, y_test
)

--- Optimizando: Regresión Logística ---
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Mejor F1-Score (CV) para Regresión Logística: 0.9341

Mejores parametros: {'clasificador__C': 1.0, 'clasificador__penalty': 'l1', 'clasificador__solver': 'saga'}

--- Optimizando: Árbol de Decisión ---
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Mejor F1-Score (CV) para Árbol de Decisión: 0.9569

Mejores parametros: {'clasificador__criterion': 'entropy', 'clasificador__max_depth': 10, 'clasificador__max_leaf_nodes': 20, 'clasificador__min_samples_leaf': 1, 'clasificador__min_samples_split': 2}

--- Optimizando: Random Forest ---
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Mejor F1-Score (CV) para Random Forest: 0.9768

Mejores parametros: {'clasificador__criterion': 'entropy', 'clasificador__max_depth': 10, 'clasificador__min_samples_leaf': 1, 'clasificador__min_samples_split': 2, 'clasificador__n_estimators': 200}

--- Optimizando: XGBoost ---


,Modelo,Mejores Parámetros,F1-Score (Train),Accuracy (Train),F1-Score (Test),Accuracy (Test)
3,XGBoost,"{'clasificador__colsample_bytree': 0.7, 'clasi...",1.000000,1.000000,0.969344,0.969231
2,Random Forest,"{'clasificador__criterion': 'entropy', 'clasif...",0.999357,0.999357,0.953934,0.953846
1,Árbol de Decisión,"{'clasificador__criterion': 'entropy', 'clasif...",0.967849,0.967866,0.938111,0.938462
6,SVM,"{'clasificador__C': 10.0, 'clasificador__gamma...",0.997429,0.997429,0.918715,0.917949
0,Regresión Logística,"{'clasificador__C': 1.0, 'clasificador__penalt...",0.957651,0.957584,0.917716,0.917949
5,MLP,"{'clasificador__activation': 'logistic', 'clas...",0.951545,0.951799,0.904537,0.905128
4,KNN,"{'clasificador__metric': 'manhattan', 'clasifi...",1.000000,1.000000,0.904069,0.905128


In [ ]:
# Llamar a la función para guardar los artefactos
# Usamos '../artefactos/' como ruta, asumiendo la estructura de carpetas
guardar_artefactos_ml(
    df_resultados=df_resultados_opt,
    pipelines=pipelines_optimizados,
    base_path='../../artefactos/' # Se recomienda crear una carpeta separada para modelos
)